In [2]:
%pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 3.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Imports
import pandas as pd
import requests
import json
import time
from lxml import etree


In [ ]:
with open('../api_creds.json') as f:
    api_keys = json.loads(f.read())
    api_key = api_keys['primary_metro_api_key']

headers = {'api_key': api_key}

In [1]:
def send_request_retry(url, headers, max_retries=3, backoff_factor=1.0):
    for attempt in range(max_retries):
        time.sleep(0.33)  # To avoid hitting API rate limits
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}. Retrying in {backoff_factor * (2 ** attempt)} seconds...")
            time.sleep(backoff_factor * (2 ** attempt))
    return response

In [10]:
# Enrich station data with the CSVs
# Create pandas dataframe from list of stations
import pandas as pd

station_list_path = '../data/api_data/station_list.xml'
station_df = pd.read_xml(station_list_path, xpath='./wm:Stations/wm:Station', namespaces={'wm': 'http://www.wmata.com'})

In [ ]:
print(station_df.head())


              Address Code        Lat LineCode1 LineCode2 LineCode3  \
0  \n                  A01  38.898303        RD      None      None   
1  \n                  A02  38.903192        RD      None      None   
2  \n                  A03  38.909499        RD      None      None   
3  \n                  A04  38.924999        RD      None      None   
4  \n                  A05  38.934703        RD      None      None   

   LineCode4        Lon                           Name StationTogether1  \
0        NaN -77.028099                   Metro Center              C01   
1        NaN -77.039766                 Farragut North             None   
2        NaN -77.043620                  Dupont Circle             None   
3        NaN -77.052648  Woodley Park-Zoo/Adams Morgan             None   
4        NaN -77.058226                 Cleveland Park             None   

   StationTogether2  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4           

In [ ]:
station_code_name_map = dict(zip(station_df['Code'], station_df['Name']))


In [13]:
print(station_code_name_map)

{'A01': 'Metro Center', 'A02': 'Farragut North', 'A03': 'Dupont Circle', 'A04': 'Woodley Park-Zoo/Adams Morgan', 'A05': 'Cleveland Park', 'A06': 'Van Ness-UDC', 'A07': 'Tenleytown-AU', 'A08': 'Friendship Heights', 'A09': 'Bethesda', 'A10': 'Medical Center', 'A11': 'Grosvenor-Strathmore', 'A12': 'North Bethesda', 'A13': 'Twinbrook', 'A14': 'Rockville', 'A15': 'Shady Grove', 'B01': 'Gallery Pl-Chinatown', 'B02': 'Judiciary Square', 'B03': 'Union Station', 'B04': 'Rhode Island Ave-Brentwood', 'B05': 'Brookland-CUA', 'B06': 'Fort Totten', 'B07': 'Takoma', 'B08': 'Silver Spring', 'B09': 'Forest Glen', 'B10': 'Wheaton', 'B11': 'Glenmont', 'B35': 'NoMa-Gallaudet U', 'C01': 'Metro Center', 'C02': 'McPherson Square', 'C03': 'Farragut West', 'C04': 'Foggy Bottom-GWU', 'C05': 'Rosslyn', 'C06': 'Arlington Cemetery', 'C07': 'Pentagon', 'C08': 'Pentagon City', 'C09': 'Crystal City', 'C10': 'Ronald Reagan Washington National Airport', 'C11': 'Potomac Yard', 'C12': 'Braddock Road', 'C13': 'King St-Old

In [ ]:
import glob
path = '../data/ridership/Metro_Rail_*.csv'
filenames = glob.glob(path)
merge_station_df = station_df[['Code', 'Name']] # , 'Lat', 'Lon', 'LineCode1', 'LineCode2', 'LineCode3', 'LineCode4']]

for filename in filenames:
    ridership_df = pd.read_csv(filename)
    #ridership_df['StationName'] = ridership_df['StationCode'].map(station_code_name_map)
    ridership_df = ridership_df.merge(merge_station_df, left_on='STOP_ID', right_on='Code')
    ridership_df.to_csv(filename+'_enriched.csv', index=False)


['../data/ridership/Metro_Rail_Ridership_Dataset_Summary_CY2024.csv', '../data/ridership/Metro_Rail_Ridership_Dataset_Summary_CY2022.csv', '../data/ridership/Metro_Rail_Ridership_Dataset_Summary_CY2023.csv', '../data/ridership/Metro_Rail_Ridership_Dataset_Summary_CYTD2025.csv']


In [7]:
# Get Bus Route JSON file
path = '../data/api_data/Bus_Routes.json'
url = 'http://api.wmata.com/Bus.svc/json/jRoutes'

response = send_request_retry(url, headers)
with open(path, 'w') as f:
    f.write(response.text)

In [16]:
# Open file and read
with open(path, 'r') as f:
    bus_data = json.load(f)
    bus_df = pd.json_normalize(bus_data, record_path='Routes')
bus_df.head()

,RouteID,Name,LineDescription
0,A11,A11 - HUNTINGTON - PENTAGON,Pentagon-Huntington
1,A12,A12 - HUNTING POINT - BALLSTON,Ballston-Hunting Point
2,A1X,A1X - BRADDOCK RD - PENTAGON CITY,Pentagon City-Braddock Rd
3,A25,A25 - LANDMARK - PENTAGON,Pentagon-S Fairlington-Landmark
4,A27,A27 - VAN DORN - PENTAGON VIA EXPRESS,Pentagon-Shirlington-Van Dorn St


In [11]:
# Load ridership DF
bus_ridership_path = '../data/ridership/BusRdr_Routes_Stops_Time_Periods.csv'
bus_ridership_df = pd.read_csv(bus_ridership_path)
bus_ridership_df.head()

,SERVICE_DAY,TIME_PERIOD,ROUTE_NAME,DIRECTION_NAME,SORT_ORDER,STOP_ID,STOP,AVG_LOAD,MAX_LOAD,SUM_PASSENGERS_ON,SUM_PASSENGERS_OFF,SUM_LOAD,LAT,LON
0,WEEKDAYS,AM Early,C11,NORTH,40,102222,OXHIPR,1.450900,3.775000,0.000000,0.000000,8.705403,38.798593,-77.000585
1,WEEKDAYS,AM Early,C11,NORTH,50,3004336,Oxon Hill P/R+Bay D,2.139164,4.650000,4.726923,0.597344,12.834982,38.798689,-77.000981
2,WEEKDAYS,AM Early,C11,NORTH,150,3000460,Indian Head Hwy/Southern Av,4.269338,6.975000,5.634615,0.727473,25.616026,38.820911,-77.000946
3,WEEKDAYS,AM Early,C11,NORTH,190,1000043,S Capitol St SE/Brandywine St SE,11.393254,17.615385,4.216392,0.247253,68.359524,38.829971,-77.007675
4,WEEKDAYS,AM Early,C11,NORTH,240,1000096,ML King Jr Av SE/#3726,15.249084,24.102564,0.625000,0.174542,91.494505,38.836819,-77.006912


In [17]:
# Join based on station id
bus_ridership_enriched = bus_ridership_df.merge(bus_df, left_on='ROUTE_NAME', right_on='RouteID')
bus_ridership_enriched.head()

,SERVICE_DAY,TIME_PERIOD,ROUTE_NAME,DIRECTION_NAME,SORT_ORDER,STOP_ID,STOP,AVG_LOAD,MAX_LOAD,SUM_PASSENGERS_ON,SUM_PASSENGERS_OFF,SUM_LOAD,LAT,LON,RouteID,Name,LineDescription
0,WEEKDAYS,AM Early,C11,NORTH,40,102222,OXHIPR,1.450900,3.775000,0.000000,0.000000,8.705403,38.798593,-77.000585,C11,C11 - NATIONAL HARBOR - L'ENFANT PLZ,S Capitol St
1,WEEKDAYS,AM Early,C11,NORTH,50,3004336,Oxon Hill P/R+Bay D,2.139164,4.650000,4.726923,0.597344,12.834982,38.798689,-77.000981,C11,C11 - NATIONAL HARBOR - L'ENFANT PLZ,S Capitol St
2,WEEKDAYS,AM Early,C11,NORTH,150,3000460,Indian Head Hwy/Southern Av,4.269338,6.975000,5.634615,0.727473,25.616026,38.820911,-77.000946,C11,C11 - NATIONAL HARBOR - L'ENFANT PLZ,S Capitol St
3,WEEKDAYS,AM Early,C11,NORTH,190,1000043,S Capitol St SE/Brandywine St SE,11.393254,17.615385,4.216392,0.247253,68.359524,38.829971,-77.007675,C11,C11 - NATIONAL HARBOR - L'ENFANT PLZ,S Capitol St
4,WEEKDAYS,AM Early,C11,NORTH,240,1000096,ML King Jr Av SE/#3726,15.249084,24.102564,0.625000,0.174542,91.494505,38.836819,-77.006912,C11,C11 - NATIONAL HARBOR - L'ENFANT PLZ,S Capitol St


In [19]:
# Save enriched bus data to df
bus_ridership_enriched.to_csv(bus_ridership_path+'_enriched.csv')